In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
data_path= '/content/drive/My Drive/INTERN_2022/DRUG/data/sensitivity_data/new_aug2/' 

In [8]:
CCLE=pd.read_csv(data_path+'CCLE_paired_common_with_gcsi.csv', index_col=0)
gCSI=pd.read_csv(data_path+'gCSI_paired_common_with_ccle.csv', index_col=0)

In [9]:
ccle_x = CCLE.iloc[:, 0:-1]
ccle_y = CCLE.iloc[:, -1]

In [10]:
gCSI = gCSI.reset_index()
gCSI = gCSI.drop(columns='index')

In [11]:
gCSI_x = gCSI.iloc[:, 0:-1]
gCSI_y = gCSI.iloc[:, -1]

In [12]:
gCSI_x.iloc[0:2, 0:-20]

,711,188,733,208,130,167,881,163,878,425,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,3.700266,5.874284,5.131369,6.823007,4.542067,5.055663,3.781692,6.743258,6.557377,-3.484916,...,13.27105,11.715869,-2.953344,-8.512375,-3.749973,-6.475866,-6.030029,-12.977965,-0.488924,0
1,2.500122,5.369221,5.603282,7.696091,4.821637,4.876002,4.059001,6.677683,6.400233,-5.159899,...,13.27105,11.715869,-2.953344,-8.512375,-3.749973,-6.475866,-6.030029,-12.977965,-0.488924,0


In [13]:
gCSI_x.iloc[:, 0:-21] = StandardScaler().fit_transform(gCSI_x.iloc[:, 0:-21])
ccle_x.iloc[:, 0:-21] = StandardScaler().fit_transform(ccle_x.iloc[:, 0:-21])


In [14]:
from sklearn.model_selection import train_test_split

gCSI_X_train, gCSI_X_test, gCSI_y_train, gCSI_y_test = train_test_split(gCSI_x, gCSI_y, test_size = 0.30, random_state = 42) 
ccle_X_train, ccle_X_test, ccle_y_train, ccle_y_test = train_test_split(ccle_x, ccle_y,test_size = 0.30, random_state = 42)

In [41]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor

# Create linear regression object
# change to RandomForestRegressor() while needed 
m1 = RandomForestRegressor(max_depth=4)#linear_model.Ridge()
m2=  RandomForestRegressor(max_depth=4)#linear_model.Ridge()

In [42]:
# in-study results: CCLE
m1.fit(ccle_X_train, ccle_y_train)

# Make predictions using the testing set
ccle_y_pred = m1.predict(ccle_X_test)


# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(ccle_y_test, ccle_y_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(ccle_y_test, ccle_y_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(ccle_y_test, ccle_y_pred))

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.84


In [43]:
#mixed study results trained on CCLE (70%)

mixed_x_test = pd.concat([ccle_X_test, gCSI_X_test], ignore_index=True)
mixed_y_test = pd.concat([ccle_y_test, gCSI_y_test], ignore_index=True)

mixed_y_pred_m1 = m1.predict(mixed_x_test)

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test, mixed_y_pred_m1))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test, mixed_y_pred_m1))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test, mixed_y_pred_m1))


Mean squared error: 0.02
Mean absolute error: 0.09
Coefficient of determination: 0.61


In [44]:


#trained on gCSI 70% data , tested on 30% gCSI

m2.fit(gCSI_X_train, gCSI_y_train)

# Make predictions using the testing set
gCSI_y_pred = m2.predict(gCSI_X_test)

# The coefficients
#print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, gCSI_y_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, gCSI_y_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, gCSI_y_pred))

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.68


In [45]:
#mixed study results trained on gCSI (70%)

mixed_y_pred_m2 = m2.predict(mixed_x_test)

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test, mixed_y_pred_m2))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test, mixed_y_pred_m2))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test, mixed_y_pred_m2))

Mean squared error: 0.02
Mean absolute error: 0.09
Coefficient of determination: 0.63


In [47]:
#mixed model m5
#train on mixed dataset and test on mixed dataset

m5=  RandomForestRegressor(max_depth= 4)#linear_model.Ridge()

mixed_x_train = pd.concat([ccle_X_train, gCSI_X_train], ignore_index=True)
mixed_y_train = pd.concat([ccle_y_train, gCSI_y_train], ignore_index=True)

m5.fit(mixed_x_train, mixed_y_train)

#make predictions on the combined test data

mixed_y_pred_m5 = m5.predict(mixed_x_test)   

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test, mixed_y_pred_m5))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test, mixed_y_pred_m5))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test, mixed_y_pred_m5))


Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.73


In [48]:
ccle_y_train_pred = m1.predict(ccle_X_train)
gCSI_y_train_pred = m2.predict(gCSI_X_train)

best that can be done on training sets

In [49]:
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_train, gCSI_y_train_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_train, gCSI_y_train_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_train, gCSI_y_train_pred))

Mean squared error: 0.01
Mean absolute error: 0.06
Coefficient of determination: 0.79


In [50]:
print("Mean squared error: %.2f" % mean_squared_error(ccle_y_train, ccle_y_train_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(ccle_y_train, ccle_y_train_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(ccle_y_train, ccle_y_train_pred))

Mean squared error: 0.01
Mean absolute error: 0.06
Coefficient of determination: 0.90


WA method

In [51]:
avg_y_pred = (ccle_y_pred + gCSI_y_pred)/2

In [52]:
avg_y_pred = pd.Series(avg_y_pred)

In [53]:
avg_y_pred_stacked = pd.concat([avg_y_pred, avg_y_pred], ignore_index=True)

In [54]:
print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test, avg_y_pred_stacked))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test, avg_y_pred_stacked))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test, avg_y_pred_stacked))

Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.72


do a linear regression on the training scores

In [55]:
m4 = linear_model.LinearRegression()

In [56]:
m4.fit(gCSI_y_train_pred.reshape(-1, 1), ccle_y_train_pred.reshape(-1,1))

# Make predictions using the testing set
ccle_y_pred_m4 = m4.predict(gCSI_y_pred.reshape(-1,1))

#avg ccle values predicted by m4 + gCSI input y values  to m4 
gCSI = (ccle_y_pred_m4 + gCSI_y_pred.reshape(-1,1))/2

#print(gCSI)
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, gCSI))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, gCSI))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, gCSI))

Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.57


In [57]:
m3 =linear_model.LinearRegression()

In [58]:
m3.fit(ccle_y_train_pred.reshape(-1, 1), gCSI_y_train_pred.reshape(-1,1))

# Make predictions using the testing set
gCSI_y_pred_m3 = m3.predict(ccle_y_pred.reshape(-1,1))

ccle = (gCSI_y_pred_m3 + ccle_y_pred.reshape(-1,1))/2
print("Mean squared error: %.2f" % mean_squared_error(ccle_y_test, ccle))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(ccle_y_test, ccle))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(ccle_y_test, ccle))

Mean squared error: 0.01
Mean absolute error: 0.09
Coefficient of determination: 0.79


In [59]:
ccle = pd.Series(ccle.squeeze())
gCSI =pd.Series(gCSI.squeeze())

In [60]:
concated = pd.concat([ccle, gCSI], ignore_index=True)

In [61]:
#test on mixed data 

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test, concated))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test, concated))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test, concated))

Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.73


cross

In [ ]:
#baseline 

# Make predictions using the testing set
ccle_y_pred_m2 = m2.predict(ccle_X_test)
print("Mean squared error: %.2f" % mean_squared_error(ccle_y_test, ccle_y_pred_m2))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(ccle_y_test, ccle_y_pred_m2))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(ccle_y_test, ccle_y_pred_m2))

Mean squared error: 0.05
Mean absolute error: 0.14
Coefficient of determination: -0.06


In [ ]:
#Baseline
# Make predictions using the testing set
gCSI_y_pred_m1 = m1.predict(gCSI_X_test)
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, gCSI_y_pred_m1))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, gCSI_y_pred_m1))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, gCSI_y_pred_m1))

Mean squared error: 0.05
Mean absolute error: 0.14
Coefficient of determination: -4.80


In [ ]:
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, ccle_y_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, ccle_y_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, ccle_y_pred))

Mean squared error: 0.05
Mean absolute error: 0.14
Coefficient of determination: -4.70


In [ ]:
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, ccle_y_pred_m3))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, ccle_y_pred_m3))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, ccle_y_pred_m3))

Mean squared error: 0.02
Mean absolute error: 0.11
Coefficient of determination: -0.87
